<a href="https://colab.research.google.com/github/VellummyilumVinoth/Train_And_Test_Using_ALBERT/blob/main/FinetunedAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.0 MB/s eta 0:00:00


In [2]:
from transformers import AlbertForMaskedLM, AlbertTokenizer

# Load the ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

In [3]:
import csv

variable_names = []
statements = []
with open('output.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        # Check if the row contains at least two columns
        if len(row) >= 2:
            # Append the variable name and source statement to their respective lists
            variable_names.append(row[0])
            statements.append(row[1])            

In [4]:
variable_names

['encodingData',
 'recordArray',
 'nextParent',
 'key',
 'arrayValues',
 'serializedArray',
 'arayIndex',
 'encoded',
 'param',
 'restOfPath',
 'headerMap',
 'stringArray',
 'headers',
 'message',
 'payload',
 'sClient',
 'message',
 'payload',
 'outputStream',
 'sClient',
 'response',
 'streamValue',
 'nextRecord',
 'personInfo',
 'profession',
 'greeting',
 'response',
 'encodedJwtComponents',
 'header',
 'payload',
 'jwtComponents',
 'decodedHeader',
 'result',
 'jsonHeader',
 'decodedPayload',
 'result',
 'jsonPayload',
 'signature',
 'header',
 'keys',
 'payload',
 'keys',
 'exp',
 'value',
 'nbf',
 'value',
 'iat',
 'value',
 'alg',
 'validatorSignatureConfig',
 'headerPayloadPart',
 'assertion',
 'signature',
 'jwksConfig',
 'certFile',
 'trustStoreConfig',
 'secret',
 'kid',
 'url',
 'clientConfig',
 'jwk',
 'publicKey',
 'signatureValidation',
 'publicKey',
 'trustStore',
 'certAlias',
 'publicKey',
 'signatureValidation',
 'sub',
 'iss',
 'aud',
 'jwtId',
 'keyId',
 'customCl

In [5]:
statements

['Encoding encodingData = encodingMap.hasKey(key) ? encodingMap.get(key) : defaultEncoding;',
 'string[] recordArray = [];',
 'string nextParent = parent + \\[\\" + key + \\"]\\";"',
 'string key = arrayName;',
 'string[] arrayValues = [];',
 'string[] serializedArray = [];',
 'int arayIndex = 0;',
 'string|error encoded = url:encode(value.toString(), \\UTF8\\");"',
 'string[] param = [];',
 'string restOfPath = string:\'join(\\\\"',
 'map<string|string[]> headerMap = {};',
 'string[] stringArray = [];',
 'map<string|string[]> headers = {};',
 'Point message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\routeguide.RouteGuide/GetFeature\\"',
 'grpc:StreamingClient sClient = check self.grpcClient->executeClientStreaming(\\routeguide.RouteGuide/RecordRoute\\");"',
 'Rectangle message;',
 'var payload = check self.grpcClient->executeServerStreaming(\\routeguide.RouteGuide/ListFeatures\\"',
 'FeatureStream outputStream = new FeatureStream(result);',
 'grpc:StreamingClient sCl

In [6]:
import re

def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += ""
    return line

source_statements = []
len_lst = []
for line in statements:
    if len(line.split(" ")) >=0:
        line = clean_text(line)
        source_statements.append(line)
        len_lst.append(len(line.split(" ")))


In [7]:
source_statements

['Encoding encodingData encodingMap hasKey key encodingMap get key defaultEncoding ',
 'string recordArray ',
 'string nextParent parent key ',
 'string key arrayName ',
 'string arrayValues ',
 'string serializedArray ',
 'int arayIndex ',
 'string error encoded url encode value toString , UTF ',
 'string param ',
 'string restOfPath string join ',
 'map string string headerMap ',
 'string stringArray ',
 'map string string headers ',
 'Point message ',
 'var payload check self grpcClient executeSimpleRPC routeguide RouteGuide GetFeature ',
 'grpc StreamingClient sClient check self grpcClient executeClientStreaming routeguide RouteGuide RecordRoute ',
 'Rectangle message ',
 'var payload check self grpcClient executeServerStreaming routeguide RouteGuide ListFeatures ',
 'FeatureStream outputStream new FeatureStream result ',
 'grpc StreamingClient sClient check self grpcClient executeBidirectionalStreaming routeguide RouteGuide RouteChat ',
 'var response check self sClient receive ',

In [8]:
from huggingface_hub import add_space_secret
import pandas as pd
import torch
import os
import re
from transformers import AdamW, Trainer, TrainingArguments, DataCollatorForLanguageModeling

import random

class VariableNamesDataset(torch.utils.data.Dataset):
    def __init__(self, variable_names, source_statements, tokenizer, mask_probability):
        self.variable_names = variable_names
        self.source_statements = source_statements
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability

    def __len__(self):
        return len(self.source_statements)
    
    def __getitem__(self, idx):
        variable_name = str(self.variable_names[idx])
        source_statement = str(self.source_statements[idx])

        print("variable names: ",len(variable_name))
        print("source statements: ",len(source_statement))

        # Tokenize the variable name and source statement
        variable_name_tokens = self.tokenizer.tokenize(variable_name)
        source_statement_tokens = self.tokenizer.tokenize(source_statement)

        print("Variable name tokens:", variable_name_tokens)
        print("Source statement tokens:", source_statement_tokens)

        # Select a variable name tokens to mask
        variable_name_indices = [i for i, token in enumerate(source_statement_tokens) if token in variable_name_tokens]

        print("Variable name indices:", variable_name_indices)

        num_to_mask = int(len(variable_name_indices)*(1 - self.mask_probability))
        indices_to_mask = random.sample(variable_name_indices, num_to_mask)

        print("Num to mask:", num_to_mask)
        print("Indices to mask:", indices_to_mask)

        # Replace the selected variable name tokens with the [MASK] token
        for i in indices_to_mask:
            source_statement_tokens[i] = '[MASK]'

        # Convert the tokens back to text
        masked_source_statement = self.tokenizer.convert_tokens_to_string(source_statement_tokens)

        # Tokenize the masked source statement
        input_ids = self.tokenizer.encode(
            masked_source_statement,
            add_special_tokens=False,
            truncation=True,
            padding='max_length',
            max_length=128
        )

        print("Input IDs:", input_ids)
        print("Masked source statement:", masked_source_statement)

        return torch.tensor(input_ids)
 

In [9]:
# Split the data into train and test sets
train_size = int(len(source_statements) * 0.8)
train_variable_names = variable_names[:train_size]
train_source_statements = source_statements[:train_size]
test_variable_names = variable_names[train_size:]
test_source_statements = source_statements[train_size:]

In [10]:
train_size

1444

In [11]:
# Prepare the data for training and testing
train_dataset = VariableNamesDataset(train_variable_names, train_source_statements, tokenizer, mask_probability = 0)

test_dataset = VariableNamesDataset(test_variable_names, test_source_statements, tokenizer, mask_probability = 1)

In [12]:
train_dataset[7]

variable names:  7
source statements:  53
Variable name tokens: ['▁encoded']
Source statement tokens: ['▁string', '▁error', '▁encoded', '▁u', 'rl', '▁encode', '▁value', '▁to', 'string', '▁', ',', '▁u', 'tf']
Variable name indices: [2]
Num to mask: 1
Indices to mask: [2]
Input IDs: [3724, 7019, 4, 287, 6362, 20523, 1923, 20, 11130, 13, 15, 287, 11720, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: string error[MASK]url encode value tostring , utf


tensor([ 3724,  7019,     4,   287,  6362, 20523,  1923,    20, 11130,    13,
           15,   287, 11720,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [13]:
test_dataset[0]

variable names:  11
source statements:  69
Variable name tokens: ['▁out', 'base', 'name']
Source statement tokens: ['▁string', '▁out', 'base', 'name', '▁check', '▁choose', 'bal', 't', 'case', 'out', 'put', 'file', 'name', '▁file', 'name', ',', '▁', 't', ',', '▁i']
Variable name indices: [1, 2, 3, 12, 14]
Num to mask: 0
Indices to mask: []
Input IDs: [3724, 70, 8436, 7259, 2631, 3538, 3817, 38, 10325, 1320, 4881, 16877, 7259, 3893, 7259, 15, 13, 38, 15, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: string outbasename check choosebaltcaseoutputfilename filename, t, i


tensor([ 3724,    70,  8436,  7259,  2631,  3538,  3817,    38, 10325,  1320,
         4881, 16877,  7259,  3893,  7259,    15,    13,    38,    15,    31,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [14]:
# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   # Evaluation and Save happens every X steps
    eval_steps = 50,                # Evaluation happens every X steps
    save_steps = 1000,               # Save checkpoint every X steps
    num_train_epochs = 10,            # Total number of training epochs
    learning_rate = 1e-5,            # Learning rate
    per_device_train_batch_size=32,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps = 500,              # Number of warmup steps
    weight_decay = 0.01,             # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=1000,              # Log every X steps
)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

trainer.train()

# Save the trained model and tokenizer
output_dir = './finetuned_albert'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


variable names:  6
source statements:  75
Variable name tokens: ['▁symbol']
Source statement tokens: ['▁bir', '▁internal', 'sym', 'bol', '▁symbol', '▁is', 'public', '▁false', ',', '▁identifier', '▁register', 'name', '▁register']
Variable name indices: [4]
Num to mask: 1
Indices to mask: [4]
Input IDs: [8719, 3117, 7261, 6966, 4, 25, 12259, 4997, 15, 25570, 2243, 7259, 2243, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: bir internalsymbol[MASK]ispublic false, identifier registername register
variable names:  6
source statements:  88
Variable name tokens: ['▁result']
Source statement tokens: ['▁un', 'sub', 'script', 'ion', 'accept', 'ed', '▁error', '▁result', '▁adapt', 

Step,Training Loss,Validation Loss
50,No log,15.098073
100,No log,13.321034
150,No log,8.433855
200,No log,7.401623
250,No log,7.402142
300,No log,7.334370
350,No log,7.221033
400,No log,7.245930
450,No log,7.007606


Streaming output truncated to the last 5000 lines.
Variable name indices: [3, 7]
Num to mask: 0
Indices to mask: []
Input IDs: [12566, 17083, 3457, 15286, 13, 14093, 2816, 15286, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: propositiongenerator generator config generator
variable names:  4
source statements:  44
Variable name tokens: ['▁prop']
Source statement tokens: ['▁proposition', '▁prop', '▁generator', '▁c', 'x', ',', '▁depth', ',', '▁rand', 's']
Variable name indices: [1]
Num to mask: 0
Indices to mask: []
Input IDs: [12566, 9476, 15286, 272, 396, 15, 5204, 15, 11310, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

('./finetuned_albert/tokenizer_config.json',
 './finetuned_albert/special_tokens_map.json',
 './finetuned_albert/spiece.model',
 './finetuned_albert/added_tokens.json')

In [17]:
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = AlbertTokenizer.from_pretrained(output_dir)

# Define a sample masked statement
masked_statement = "boolean [MASK] = True;"

# Tokenize the masked statement
input_ids = tokenizer.encode(masked_statement, add_special_tokens=True, return_tensors='pt')

# Find the position of the masked token
masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

# Generate predictions for the masked token
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores
probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k = torch.topk(probs, k=5)

# Print the top 5 predictions and their probability scores
for i, idx in enumerate(top_k.indices):
    token = tokenizer.convert_ids_to_tokens([idx])[0]
    print(f"Prediction {i+1}: {token}, Probability: {top_k.values[i].item()}")


Prediction 1: ▁identifier, Probability: 0.0391300693154335
Prediction 2: ▁binary, Probability: 0.013942180201411247
Prediction 3: ▁error, Probability: 0.012185533531010151
Prediction 4: type, Probability: 0.011171799153089523
Prediction 5: ▁value, Probability: 0.01116679236292839


In [18]:
# without fine tune the ALBERT model 

from transformers import pipeline

unmasker = pipeline('fill-mask', model='albert-base-v2')

unmasker( "boolean [MASK] = True;")

[{'score': 0.17232464253902435,
  'token': 29833,
  'token_str': 'evalle',
  'sequence': 'boolean evalle = true;'},
 {'score': 0.023911982774734497,
  'token': 28153,
  'token_str': 'joyah',
  'sequence': 'boolean joyah = true;'},
 {'score': 0.015405314974486828,
  'token': 28273,
  'token_str': 'parentheses',
  'sequence': 'boolean parentheses = true;'},
 {'score': 0.007617090828716755,
  'token': 12894,
  'token_str': '//',
  'sequence': 'boolean // = true;'},
 {'score': 0.006087141111493111,
  'token': 4274,
  'token_str': '2:',
  'sequence': 'boolean 2: = true;'}]